In [ ]:
## Distributed package # works
using Distributed
n = Int(1e8)

@time @sync nheads1 = @distributed (+) for i = 1:n; rand(Bool); end

nheads2 = 0;
@time for i = 1:n; global nheads2+=rand(Bool); end
println(nheads2)

## Show what worker that is involved
n = 32
a = zeros(n);
Threads.@threads for i = 1:n
    a[i] = Threads.threadid()
end
println(a)

# Find out number of threads
Threads.nthreads()

# Distribute workers within for loop (Experimental only!)

n = Int(1e8)

sum = 0
@time Threads.@threads for i = 1:n
    global sum
    sum+=rand(Bool)
end
println(sum)

sum = 0
@time for i = 1:n
    global sum
    sum+=rand(Bool)
end
println(sum)

In [1]:
#addprocs(1)
#rmprocs(workers())
#addprocs(7); # add number of worker processes
# Trying to map out distribution
using Distributed
@everywherebasepath = "/home/dreuter/Github/julia-paths"
@everywherethispath = "/LearningPath-Julia/"
@everywherefullpath = string(basepath,thispath)
@everywhere include(string(fullpath,"Counter.jl"))

LoadError: syntax: unexpected "="

In [2]:
function parallel_map(n,n_par)
    joblist = []
    n_job = Int(round(n/n_par))
    for i = 1:n_par
        push!(joblist, @spawn(Counter.count_heads(n_job)))
    end
    return joblist
end

parallel_map (generic function with 1 method)

In [3]:
n = Int(1e12);
@time joblist = parallel_map(n,16)

  1.009722 seconds (1.49 M allocations: 74.433 MiB, 6.90% gc time)


16-element Array{Any,1}:
 Future(1, 1, 1, nothing) 
 Future(1, 1, 2, nothing) 
 Future(1, 1, 3, nothing) 
 Future(1, 1, 4, nothing) 
 Future(1, 1, 5, nothing) 
 Future(1, 1, 6, nothing) 
 Future(1, 1, 7, nothing) 
 Future(1, 1, 8, nothing) 
 Future(1, 1, 9, nothing) 
 Future(1, 1, 10, nothing)
 Future(1, 1, 11, nothing)
 Future(1, 1, 12, nothing)
 Future(1, 1, 13, nothing)
 Future(1, 1, 14, nothing)
 Future(1, 1, 15, nothing)
 Future(1, 1, 16, nothing)

## Compare with and without parallell jobs

In [4]:
@time sum(map(fetch, joblist))

MethodError: MethodError: no method matching +(::RemoteException, ::RemoteException)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

In [ ]:
@time Counter.count_heads(n)